In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

#### Lab 5: Visualizations and Comparisons

Welcome to lab 5! This week, we will go over creating various visualizations and using conditional statements.

The relevant sections in the books are
* [Chapter 6](https://www.inferentialthinking.com/chapters/06/4/Example_Gender_Ratio_in_the_US_Population.html) for computing proportions
* [Chapter 7](https://www.inferentialthinking.com/chapters/07/Visualization.html) for `barh` and `hist`, 
* [Section 8.2](https://www.inferentialthinking.com/chapters/08/2/Classifying_by_One_Variable.html) for `group`

You will also get to retrieve source data from a Wikipedia page and do some clean up to get it ready for analysis. *Acknowledgements:* The exercise on collecting data about presidential elections has been adopted from the [UCSB DS100 homework](https://ucsb-ds.github.io/s20/).

In this lab you will be using
* Boolean operators
* functions that you defined yourself
* `group` to transform the tables
* `where` to select the relevant rows
* `column` to get the needed column as an array
* `barh` and `hist` for visualizing data


First, set up the tests and imports by running the cell below.

In [1]:
import numpy as np
from datascience import *

# These lines set up graphing capabilities.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# 0. Group partners

For this lab, you are allowed to work with up to 2 other students. Each of you should do the work in your own notebook, but you can brainstorm and work together on the exploratory questions. 

If you didn't attend the lab and are not assigned to a group, post on Piazza with your meeting time availability and see if any of the groups will have you join them. If you cannot find a group, work solo or attend the office hours to work with one of the mentors.

In the following cell, list your name and the names of all lab partners (if applicable).

In [2]:
lab_partners = Table().with_columns(
    "Name", make_array("name"),
    "Email", make_array("@ucsb.edu")
)
lab_partners

# 1. Analyzing the U.S. Elections

It is the Election Season!

This year (2020), the citizens of the United States will be electing their 46th president. Let us look at how states voted in presidential elections between 1972 and 2016.

## 1.1: Get the source data

Unlike prior assignments, we're going to make you go **get the data yourself**. Specifically, we'd like you to use this table from wikipedia: [https://en.wikipedia.org/wiki/List_of_United_States_presidential_election_results_by_state](https://en.wikipedia.org/wiki/List_of_United_States_presidential_election_results_by_state). 

You can convert the table into csv format using this website: [https://wikitable2csv.ggor.de/](https://wikitable2csv.ggor.de/). Simply paste the address (URL) of the website into wikitable2csv and leave the default options as they are.

On the next page, click "Download" on **Table 1**: you should download a file called `table-1.csv`.

Upload this file to your [Jupyterhub folder for this lab](./) (where this ipynb is) and rename the csv to "`presidential_elections.csv`". Then run the cell below to make sure that you did everything properly.

<!--
BEGIN QUESTION
name: q1_1
points: 1
manual: false
-->

In [3]:
election = Table().read_table("presidential_elections.csv")
election

In [ ]:
grader.check("q1_1")

## 1.2: Clean the data

The data in this table is pretty messy. We need to create a clean version of this table called `election_1972_to_2016`. It should contain exactly 51 rows (corresponding to the 50 state plus Washington DC) and 13 columns (one for the "State" and one for each of the election year from 1972 to 2016, to include only republican `R` and democratic `D` votes in each state).

Let's take a look at the column labels to see if we need to use all columns and whether we should rename any of them.

<!--
BEGIN QUESTION
name: q1_2
points: 1
manual: false
-->

In [8]:
### Get the column names
column_names = ...
column_names

In [ ]:
grader.check("q1_2")

Looks like we have some cleaning to do!

Since we want to end up with 13 columns (one for the state and one for each of the election year from 1972 to 2016), we can extract the first and the last 14 columns of the table and then remove the columns "Unnamed: 60" and "State.1". We should also remove the weird symbol in the "2000" and "2016" columns to make it easier to work with the columns.

Let's do this in stages, saving each intermediate step into different variables:
1. to extract the last $K$ columns, let's go ahead and drop the first $N$ columns, skipping the first one (where $K=14$ and $N=66-K-1 = 51$);
1. drop the column `'Unnamed: 60'` and the `'State.1'`;
1. rename the `'2000 ‡'` and `'2016 ‡'` columns to remove the non-numerical symbol.


### 1.2.1: `drop` unnecessary columns (step 1)

As always, we recommend referring to the documentation if you don't remember how to use the [`drop` method](http://data8.org/datascience/_autosummary/datascience.tables.Table.drop.html#datascience.tables.Table.drop) with the tables. Note that in the documentation it says that `drop` has a parameter `column_or_columns`, which must be an existing header name, or a valid column index.

First, let's just get the indices of columns that we want to drop. We'll use them in the next step as an input into the `drop` method.

Since it is too tedious to list the columns by name, let's use the fact that we can give it a list with the column indices. Let's use `np.arange` to generate the indices of the 51 columns: we want to start from the second column (to preserve the first,"State", column), i.e., our indices start at 1 and go up to 51.

<!--
BEGIN QUESTION
name: q1_2_1a
points: 1
manual: false
-->

In [12]:
column_indices_to_drop = ...

In [ ]:
grader.check("q1_2_1a")

Now, use the array `column_indices_to_drop` that you just created as an input into the `drop()` method.

<!--
BEGIN QUESTION
name: q1_2_1b
points: 1
manual: false
-->

In [16]:
election_drop52 = ...
election_drop52

In [ ]:
grader.check("q1_2_1b")

### 1.2.2: `drop` unnecessary columns (step 2)

In this step, since we only need to remove the columns `'Unnamed: 60'` and the `'State.1'`, we can give the column names directly to the `drop` method. 

*Hint:* we shouldn't be dropping the columns from the original `election` table, since we want to continue cleaning the table that we got in the previous step.

<!--
BEGIN QUESTION
name: q1_2_2
points: 1
manual: false
-->

In [21]:
election_all_dropped = ...

In [ ]:
grader.check("q1_2_2")

### 1.2.3: clean the column names (step 3)

Since we needed **the column names** to contain **only** the numerical values, we need to rename the `'2000 ‡'` and `'2016 ‡'` columns to remove the non-numerical symbol.

Once again, you can use [the documentation](http://data8.org/datascience/_autosummary/datascience.tables.Table.relabeled.html#datascience.tables.Table.relabeled) to check how to rename the column.

**Note:** Make sure to **not** change values in the `election_all_dropped` table; **return a new table** instead.

<!--
BEGIN QUESTION
name: q1_2_3
points: 1
manual: false
-->

In [24]:
election_1972_to_2016 = ...
election_1972_to_2016

In [ ]:
grader.check("q1_2_3")

### 1.2.4: final clean-up

Now, there's still something that's off about our table. Since we just cleaned our columns and can verify that we have the correct number of them and the correct labels, there's only one other aspect of the table that we should check. What do you think it is? Where should you look?

*Hint:* Remember that you can see the entire table by using `.show()`.

*Hint:* Consider using `where` in your solution to exclude what you don't want.

<!--
BEGIN QUESTION
name: q1_2_4
points: 1
manual: false
-->

In [28]:
election_1972_to_2016_final = ...
election_1972_to_2016_final

In [ ]:
grader.check("q1_2_4")

Whew! We are done with the clean-up! 
We hope that it makes you appreciate the nicely clean datasets that we've worked with so far. 
We also hope that you are pretty happy that you could use the skills you've learned in this class to clean a real-world dataset.

We are now ready to...


## 1.3: Visualize the data

For each election year from 1972 to 2016, we can see how many states voted republican `R` or democratic `D` vote.


### 1.3.1: Grouping by year

Let's see how distribution of state votes has changed for the past 3 elections in 2008, 2012, and 2016.

In order to do it, we need to count how many republican `R` or democratic `D` votes we have in the corresponding column. To do so, we can use the Table's `group()` method, which allows us to count how frequently each vote appears in the table.

> The `group()` method takes as its argument the *label of the column* that contains the categories, and returns a table of *counts of rows* in each category.

(*Side note: the column of counts that is returned by the `group()` is always called `'count'`*.)

In [32]:
election_2008_counts = election_1972_to_2016_final.group('2008')
election_2008_counts

Now, in order to visualize the counts in a bar graph, we use the `barh` method with the table that we created above.

In [33]:
election_2008_counts.barh('2008')

#### Method chaining

> A lot of times, we just want to take a quick peek at the distribution and don't want to first store the counts in a separate table in order to visualize them. This is when the method-chaining comes in handy: instead of storing the results of the `group` method in a variable and then using that variable in the next method/command, we can pass the table that is generated by the `group` method to the next method (and the next and the next and so on, if we wanted to).

Note that the below expression is equivalent to the two method that we ran above. Here, we just chained `group` and `barh`, so the table with the counts that resulted from using `group` was sent to `barh` for visualization.

In [34]:
election_1972_to_2016_final.group('2008').barh('2008')

OK, you should now be equipped with everything you need to check the distribution of votes in 2012 and 2016.

In [35]:
### election_1972_to_2016_final.group('...').barh('...')

Do you think that the change in the vote distribution between 2008 and 2012 was indicative of the trend that led to the 2016 results?

In [36]:
### Try to write this command from scratch for the year 2016
election_1972_to_2016_final

### 1.3.2: `group`ing by multiple years

For each election year from 1972 to 2016, we can see how many times a state voted republican `R` or democratic `D` vote. However, because that data is stored in rows, in order to visualize the distribution for a state, we first need to extract its row.

Let's take a look at California.

In [37]:
ca_votes = election_1972_to_2016_final.where('State', 'California')
ca_votes

Note that up until 1992, California voted Republican. In 1992, the United States elected their [42nd president](https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States), which was the first democratic president in 12 years.

We can take a look at other states to see how they voted since 1972.

In [38]:
election_1972_to_2016_final.where('State', 'Hawaii')

What if we wanted to know how many states voted the same for the past 3 elections?

In order to answer this question, we cannot group the rows just by the value in the single column, i.e., a single year, like we did in the previous question. We need to group them by the values in 3 columns, which correspond to the 3 elections that we are interested in.

To do that, we need to create an array that contains the labels of the columns that we want to group.

In [39]:
election_1972_to_2016_final.group(make_array('2008', '2012', '2016'))

From the result above, we see that 21 state voted democratic and 22 states voted republican for the past 3 elections.

Are you curious to find out which 6 states voted republican in 2016 after voting democratic in 2008 and 2012? Let's find out!

### 1.3.3: Extracting specific values

To answer this question, we can use the method chaining to extract the rows for the states that voted `D` in 2008 and 2012 and then voted `R` in 2016.

In [40]:
election_1972_to_2016_final.where('2008', 'D').where('2012', 'D').where('2016', 'R')

If we wanted only a list of the states, then we could run the above command and extract just the names and store them in an array using the `column` method:

In [41]:
election_1972_to_2016_final.where('2008', 'D').where('2012', 'D').where('2016', 'R').column("State")

Note that the following is equivalent to what we just did above: `ddr_2008_2016` is the name that we are giving to the table that we got from chaining the `where` methods, and we can then use that table with the `.column("State")` to extract the state names.

<!--
BEGIN QUESTION
name: q1_3_3a
points: 1
manual: false
-->

In [42]:
ddr_2008_2016 = election_1972_to_2016_final.where('2008', 'D').where('2012', 'D').where('2016', 'R')
# Now, extract the state names using ddr_2008_2016
ddr_states_2008_2016 = ...
ddr_states_2008_2016

In [ ]:
grader.check("q1_3_3a")

Your turn.

Find the names of the two states that since voted `R` after voting `D` in 2008.

<!--
BEGIN QUESTION
name: q1_3_3b
points: 1
manual: false
-->

In [44]:
drr_2008_2016 = ...
drr_states_2008_2016 = ...
drr_states_2008_2016

In [ ]:
grader.check("q1_3_3b")

<!-- BEGIN QUESTION -->

## 1.4: Your Exploration

This is an optional section which gives you a chance to experiment with the data and the methods. We will take a look at what you did here, and if you had interesting questions that you were able to ask and answer, we will award extra points for your explorations.

We recommend for you to finish the lab and then come back to this section. Feel free to use the `Insert` menu above to add extra cells. Remember to include a Markdown cell in which you write down the question(s) that you are attempting to answer with your code.

If you decide to (go above and beyond, and) use additional datasets, you definitely may include them in your analysis. Make sure you do the following:
* include the URL for the source of the data
* explain why you needed it / what part of the data you want to use
* if the data needed cleaning but the steps to clean the data are not in the notebook, describe what you did to clean the columns/rows.
* submit the additional CSV file with your notebook to Gradescope.

Have fun!

<!--
BEGIN QUESTION
name: q1_4
points: 1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## 2. Our Class Data

As you may remember, PA02 asked you to fill out a survey about your experiences. It also allowed us to collect fun data to use as we practice using Table methods.

Let's go ahead and import the files.

In [47]:
majors = Table().read_table("ds1-survey-major.csv")
os = Table().read_table("ds1-survey-os.csv")
first_gen = Table().read_table("ds1-survey-first-gen.csv")
job_charity = Table().read_table("ds1-survey-pronouns-charity-job.csv")
experience = Table().read_table("ds1-survey-pronouns-experience.csv")

<!-- BEGIN QUESTION -->

### 2.1: Randomization

Note that we have data split across multiple files. We have shuffled the order of the rows, so the responses in each row of the files do not belong to the same student. What are possible reasons why you think we did this randomization?

<!--
BEGIN QUESTION
name: q2_1
points: 1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

### 2.2: Majors

### 2.2.1: Counting Majors

Let's take a look at how many majors we have represented in this class. Which major has the most students represented on the survey?

To answer this question,
1. count how many students from each major took the survey (using `group`, i.e., `group('Major')`),
1. sort the counts in the *resulting table*, so that the major with the most students is in the first row (i.e., `descending = True`)

<!--
BEGIN QUESTION
name: q2_2a
points: 1
manual: false
-->

In [48]:
sorted_majors = ...

In [ ]:
grader.check("q2_2a")

<!-- BEGIN QUESTION -->

Now that we have a nicely sorted table, let's create a bar graph (using `barh`) that shows the distribution of majors.

<!--
BEGIN QUESTION
name: q2_2b
points: 1
manual: true
-->

In [50]:
...

<!-- END QUESTION -->

### 2.2.2: Thresholding Majors

We would like to recruit more students from the majors that currently have only 1 student in our class. How do we get the list of those majors?

*Hint:* This is very similar to what you did earlier in the lab: it involves `where` and `column`.

<!--
BEGIN QUESTION
name: q2_2c
points: 1
manual: false
-->

In [51]:
majors_to_reach = ...

In [ ]:
grader.check("q2_2c")

### 2.3: Analyzing expriences of First-Gen students

#### 2.3.1: Counting first-gen students
Let's see if we have enough first-gen students in the class to do any type of reasonable analysis.

<!--
BEGIN QUESTION
name: q2_3a
points: 1
manual: false
-->

In [54]:
first_gen_count = ...
first_gen_count

In [ ]:
grader.check("q2_3a")

#### 2.3.2: Computing proportions

Add a column called `"proportion"` that shows th proportion of each type of student; store the result in a new table `first_gen_prop`. 

*Hint:* Feel free to refer to [Chapter 6](https://www.inferentialthinking.com/chapters/06/4/Example_Gender_Ratio_in_the_US_Population.html) for an example of how to do it.

<!--
BEGIN QUESTION
name: q2_3b
points: 1
manual: false
-->

In [57]:
count_column = ...
total_students = ...

first_gen_prop = first_gen_count.with_column("proportion", count_column/total_students).set_format('proportion', PercentFormatter)
first_gen_prop

In [ ]:
grader.check("q2_3b")

#### 2.3.4: Prior experience

It is a common occurrence that first-gen students generally have less computing experience than those whose parents have gone to college.

Let's group the table by two columns `"First gen", "Prior experience"` and see if the above statement holds true in our class.

<!--
BEGIN QUESTION
name: q2_3c
points: 1
manual: false
-->

In [61]:
first_gen_experience_count = ...

In [ ]:
grader.check("q2_3c")

## 2.4: Visualizing charity contributions

Two of the questions on the survey were:

> (Charity) Suppose you won a million dollars and were given the choice to either spend it on yourself or give it away to save the lives of children who would otherwise die of starvation. The more money you gave, the more children would be saved (let's say each dollar saved a child's life). What percent of the money would you give away?

and

> Would you rather have a job you hate that makes 500,000 dollars a year or a job you love that makes 100,000 dollars?

Below are the results, which include the pronouns of the respondent.

In [64]:
job_charity

<!-- BEGIN QUESTION -->

### 2.4.1: Binning the results

Let's create a histogram that shows the distribution of the charitable contribution among the survey respondents.

<!--
BEGIN QUESTION
name: q2_4a
points: 1
manual: true
-->

In [65]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 2.4.2: Empty bins

Notice the gap that we see in the bins. Which percentages of charity donation does this gap correspond to?

<!--
BEGIN QUESTION
name: q2_4b
points: 1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### 2.4.3: How many altruists?

Does the histogram above capture the maximum possible contribution? How do you know? How can you use the `.bin()` method to verify that the histogram contains the values for the people who would have given away all of their winning? See [Chapter 7.2](https://www.inferentialthinking.com/chapters/07/2/Visualizing_Numerical_Distributions.html) if you need to look up how to use the method and what parameters it takes.

In [66]:
...

Let's use `where` to see if we can count **how many people** decided to give away 100 percent of their winnings.

<!--
BEGIN QUESTION
name: q2_4_3a
points: 1
manual: false
-->

In [67]:
gave_away_all = ...

In [ ]:
grader.check("q2_4_3a")

**How many** of those people use the pronouns `"he/him/his"`?

*Hint:* Unless you saved the result of the `where` statement that you used to answer the previous question, you will need two `where` conditions.

<!--
BEGIN QUESTION
name: q2_4_3b
points: 1
manual: false
-->

In [70]:
gave_away_all_guys = ...

In [ ]:
grader.check("q2_4_3b")

<!-- BEGIN QUESTION -->

### 2.4.4: Interpreting the histogram

A lot of times when you come across a histogram, you don't have access to the original data. This is when it becomes important to know how to interpret the histogram and the density that it displays.

Read about in [Chapter 7.2](https://www.inferentialthinking.com/chapters/07/2/Visualizing_Numerical_Distributions.html#The-Vertical-Axis:-Density-Scale) and then come back to this question.

Follow the example in [Chapter 7.2](https://www.inferentialthinking.com/chapters/07/2/Visualizing_Numerical_Distributions.html#The-Vertical-Axis:-Density-Scale) to compute the percent and the total number of people who would have donated 40-60 percent of their winning (not including 60).

<!--
BEGIN QUESTION
name: q2_4_4
points: 1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## 2.5: Your Exploration

This is an optional section which gives you a chance to experiment with the other data tables that we provided. 
Take a look at whether the respondents who donate more to charity generally select the jobs that they love, whether confidence in programming has any relation to lecture viewing preferences, and write down any other questions that you want to answer using these datasets.

We will take a look at what you did here, and if you had interesting questions that you were able to ask and answer, we will award extra points for your explorations.

We recommend for you to finish the lab and then come back to this section. Feel free to use the `Insert` menu above to add extra cells. Remember to include a Markdown cell in which you write down the question(s) you are attempting to answer with your code.

<!--
BEGIN QUESTION
name: q1_4
points: 1
manual: true
-->

<!-- END QUESTION -->

Congratulations, you're done with Lab 5!  Be sure to 
- **save the notebook** first (**`Save and Checkpoint`** from the `File` menu)
- go up to the `Kernel` menu and select `Restart & Clear Output` (make sure the notebook is saved first, because otherwise, you will lose all your work!). 
* go to `Cell -> Run All`. Carefully look through your notebook and verify that all computations execute correctly. You should see **no errors**; if there are any errors, make sure to correct them before you submit the notebook.
* go to `File -> Download as -> Notebook` and download the notebook to your own computer. ([Please verify](https://ucsb-ds.github.io/ds1-f20/troubleshooting/#i-downloaded-the-notebook-file-but-it-saves-as-the-ipynbjson-extension-so-whenever-i-upload-it-to-gradescope-it-fails) that it got saved as an .ipynb file.)
* Upload the notebook to [Gradescope](https://www.gradescope.com/). **If you used additional CSV files, remember to upload them with your notebook.** You can drag and drop both files or hold down Ctrl to click on multiple files when you are uploading them.


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export("lab05.ipynb")